In [9]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# data_100k = pd.read_csv('blackjack_simulator.csv').sample(n=100000, random_state=21)
data_500k = pd.read_csv('blackjack_simulator.csv').sample(n=500000, random_state=21)
# data_1m = pd.read_csv('blackjack_simulator.csv').sample(n=1000000, random_state=21)

# data_100k.to_csv('./blackjack_sample_100k.csv')
data_500k.to_csv('./blackjack_sample_500k.csv')
# data_1m.to_csv('./blackjack_sample_1m.csv')

data_500k.head()

,shoe_id,cards_remaining,dealer_up,initial_hand,dealer_final,dealer_final_value,player_final,player_final_value,actions_taken,run_count,true_count,win
33791619,556105,393,4,"[4, 7]","[4, 3, 10]",17,"[[4, 7, 10]]",[21],[['D']],-5,0,2.0
19602821,322598,389,9,"[2, 10]","[9, 10]",19,"[[2, 10, 7]]",[19],"[['H', 'S']]",3,0,0.0
34664548,570473,268,10,"[7, 6]","[10, 4, 10]",24,"[[7, 6, 2, 10]]",[25],"[['H', 'H']]",15,2,-1.0
37395938,615427,246,10,"[4, 9]","[10, 10]",20,"[[4, 9, 11, 9]]",[23],"[['H', 'H']]",-2,0,-1.0
10736666,176685,136,7,"[10, 9]","[7, 10]",17,"[[10, 9]]",[19],[['S']],6,2,1.0


In [10]:
data2 = pd.read_excel('./blackjack cleaner.xlsx', sheet_name='blackjack_sample_500k')


data2.describe()

,dealer_up,player_card_1,player_card_2,player_card_3,player_card_4,player_card_5,win/loss
count,487743.000000,487743.000000,487743.000000,221187.000000,61392.000000,11360.000000,487743.000000
mean,7.345348,7.310001,7.308341,7.311515,7.337161,7.307570,0.503191
std,2.913487,2.915530,2.921654,2.921636,2.905504,2.902278,0.499990
min,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,0.000000
25%,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,0.000000
50%,8.000000,8.000000,8.000000,8.000000,8.000000,8.000000,1.000000
75%,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,1.000000
max,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,1.000000


In [11]:
data2.head()

,dealer_up,player_card_1,player_card_2,player_card_3,player_card_4,player_card_5,player_move_1,player_move_2,player_move_3,player_move_4,win/loss
0,4,4,7,10.0,NaN,NaN,D,NaN,NaN,NaN,1
1,9,2,10,7.0,NaN,NaN,H,S,NaN,NaN,1
2,10,7,6,2.0,10.0,NaN,H,H,NaN,NaN,0
3,10,4,9,11.0,9.0,NaN,H,H,NaN,NaN,0
4,7,10,9,NaN,NaN,NaN,S,NaN,NaN,NaN,1


In [12]:
# data2.columns


model_1_data = data2[['dealer_up', 'player_card_1', 'player_card_2', 'player_move_1', 'win/loss']].dropna(axis=0)
model_2_data = data2[['dealer_up', 'player_card_1', 'player_card_2', 'player_card_3', 'player_move_1', 'player_move_2', 'win/loss']].dropna(axis=0)
model_3_data = data2[['dealer_up', 'player_card_1', 'player_card_2', 'player_card_3', 'player_card_4', 'player_move_1', 'player_move_2', 'player_move_3', 'win/loss']].dropna(axis=0)

#['dealer_up', 'player_card_1', 'player_card_2', 'player_card_3',
#        'player_card_4', 'player_card_5', 'player_move_1', 'player_move_2',
#        'player_move_3', 'player_move_4', 'win/loss']

In [13]:
#getdummies
model_1_data = pd.get_dummies(model_1_data, columns=['dealer_up', 'player_card_1', 'player_card_2', 'player_move_1'])
model_2_data = pd.get_dummies(model_2_data, columns=['dealer_up', 'player_card_1', 'player_card_2', 'player_card_3', 'player_move_1', 'player_move_2'])
model_3_data = pd.get_dummies(model_3_data, columns=['dealer_up', 'player_card_1', 'player_card_2', 'player_card_3', 'player_card_4', 'player_move_1', 'player_move_2', 'player_move_3'])


In [14]:
#create X and Y variables for Random Forest
X1 = model_1_data.drop('win/loss', axis=1)
X2 = model_2_data.drop('win/loss', axis=1)
X3 = model_3_data.drop('win/loss', axis=1)

y1 = model_1_data['win/loss']
y2 = model_2_data['win/loss']
y3 = model_3_data['win/loss']

In [15]:
#train test split data fror all 3 random forest models
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, random_state=21)
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, random_state=21)
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, random_state=21)

In [16]:
#initialize all 3 random forest models.
rf_model_1 = RandomForestClassifier(n_estimators=500, random_state=21)
rf_model_2 = RandomForestClassifier(n_estimators=500, random_state=21)
rf_model_3 = RandomForestClassifier(n_estimators=500, random_state=21)


In [17]:
#fit all 3 random forest models
rf_model_1 = rf_model_1.fit(X1_train, y1_train)
rf_model_2 = rf_model_2.fit(X2_train, y2_train)
rf_model_3 = rf_model_3.fit(X3_train, y3_train)

In [18]:
#predict moves for all 3 models
predictions_1 = rf_model_1.predict(X1_test)
predictions_2 = rf_model_2.predict(X2_test)
predictions_3 = rf_model_3.predict(X3_test)

In [19]:
acc_score_1 = accuracy_score(y1_test, predictions_1)
acc_score_2 = accuracy_score(y2_test, predictions_2)
acc_score_3 = accuracy_score(y3_test, predictions_3)

print(acc_score_1, acc_score_2, acc_score_3)

0.6932384401441883 0.6934522103090419 0.7087116090399845


In [21]:
#classification report 1
cr1 = classification_report(y1_test, predictions_1)
print(cr1)

              precision    recall  f1-score   support

           0       0.66      0.76      0.71     57807
           1       0.74      0.63      0.68     61204

    accuracy                           0.69    119011
   macro avg       0.70      0.70      0.69    119011
weighted avg       0.70      0.69      0.69    119011



In [22]:
#classification report 2
cr2 = classification_report(y2_test, predictions_2)
print(cr2)

              precision    recall  f1-score   support

           0       0.66      0.75      0.70     16679
           1       0.74      0.64      0.68     18203

    accuracy                           0.69     34882
   macro avg       0.70      0.70      0.69     34882
weighted avg       0.70      0.69      0.69     34882



In [23]:
#classification report 3
cr3 = classification_report(y3_test, predictions_3)
print(cr3)

              precision    recall  f1-score   support

           0       0.66      0.68      0.67      4557
           1       0.75      0.73      0.74      5797

    accuracy                           0.71     10354
   macro avg       0.70      0.71      0.71     10354
weighted avg       0.71      0.71      0.71     10354

